### notebook purpose

- build tools for efficiently comparing across multiple directories

In [27]:
import os, sys
from os import listdir
from os.path import isfile, join, isdir
import numpy as np

sys.path.append('/home/vanveen/ConvDecoder/')
from utils.evaluate import calc_metrics

In [4]:
path_base = '/bmrNAS/people/dvv/out_qdess/accel_4x/echo_joint/'

path_1 = path_base + 'mask_rect/'
path_2 = path_base + 'mask_square/'
path_gt = path_base + 'gt/'

In [33]:
def get_file_list(path):
    files = [f for f in listdir(path) if isfile(join(path, f))]
    files.sort()
    return files

def get_mtr_ids(file_list):
    ''' given list of files 
        return list of unique XXX entries in MTR_XXX_... ''' 
    mtr_ids = list(set([s.split('MTR_')[1][:3] for s in file_list]))
    mtr_ids.sort()    
    return mtr_ids

def get_mtr_ids_and(path_1, path_2):
    ''' given two paths, return list of matching filenames in each directory '''
    
    # get list of mtr ids in each directory
    mtr_ids_1 = get_mtr_ids(get_file_list(path_1))
    mtr_ids_2 = get_mtr_ids(get_file_list(path_2))
    
    mtr_ids_and = [a and b for a, b in zip(mtr_ids_1, mtr_ids_2)]
    mtr_ids_and.sort()
    
    if len(mtr_ids_and) == 0:
        raise ValueError('no overlapping files between paths')
        
    return mtr_ids_and

def load_imgs(mtr_id_list, path):
    ''' given list of mtr_ids and a directory 
        return single array w all imgs '''
    
    # indicator string if loading gt
    gt_str = '_gt' if '/gt/' in path else '' 

    num_samps = len(mtr_id_list)
    num_echos, num_y, num_z = 2, 512, 160
    arr = np.empty((num_samps, num_echos, num_y, num_z))
        
    for idx_s, mtr_id in enumerate(mtr_id_list):
        
        arr[idx_s, 0] = np.load('{}MTR_{}_e1{}.npy'.format(path, mtr_id, gt_str))
        arr[idx_s, 1] = np.load('{}MTR_{}_e2{}.npy'.format(path, mtr_id, gt_str))
        
    return arr

def calc_metrics_imgs(imgs_gt, imgs_out):
    ''' given (gt, out) imgs over many samples / echos
        compute metrics for each '''
    
    assert imgs_gt.shape == imgs_out.shape
    assert imgs_gt.shape[-2:] == (512, 160)
    assert imgs_gt.ndim == 4
    
    num_samps = imgs_gt.shape[0]
    num_echos = imgs_gt.shape[1]
    num_metrics = 4 # vif, msssim, ssim, psnr
    
    metrics = np.empty((num_samps, num_echos, num_metrics))
    
    for idx_s in np.arange(num_samps):
        for idx_e in np.arange(num_echos):
            
            im_gt, im_out = imgs_gt[idx_s, idx_e], imgs_out[idx_s, idx_e]
            
            metrics[idx_s, idx_e] = np.array(calc_metrics(im_gt, im_out))
            
    return metrics

# TODO

- x

In [34]:
mtr_id_list = get_mtr_ids_and(path_1, path_2)

imgs_gt = load_imgs(mtr_id_list, path=path_gt)
imgs_rect = load_imgs(mtr_id_list, path=path_1)
imgs_square = load_imgs(mtr_id_list, path=path_2)

In [35]:
metrics_square = calc_metrics_imgs(imgs_gt, imgs_square)
metrics_rect = calc_metrics_imgs(imgs_gt, imgs_rect)

/home/vanveen/heck/lib/python3.8/site-packages/pytorch_msssim/ssim.py:50: UserWarning: Skipping Gaussian Smoothing at dimension 2+1 for input: torch.Size([1, 1, 32, 10]) and win size: 11
  warnings.warn(


In [39]:
mtr_id_list

['005',
 '006',
 '030',
 '034',
 '048',
 '052',
 '065',
 '066',
 '080',
 '096',
 '099',
 '120']

In [44]:
metrics_diff = metrics_square - metrics_rect

metrics_diff > 0

array([[[False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False]],

       [[ True, False, False, False],
        [ True, False, False, False]],

       [[ True, False, False, False],
        [False, False, False, False]],

       [[ True, False, False, False],
        [ True, False, False, False]],

       [[False,  True,  True,  True],
        [False, False,  True, False]],

       [[ True, False, False,  True],
        [ True, False,  True, False]],

       [[ True,  True,  True,  True],
        [ True,  True,  True,  True]],

       [[ True, False, False, False],
        [ True, False,  True,  True]],

       [[ True, False, False,  True],
        [False, False, False, False]],

       [[ True, False, False, False],
        [ True, False, False, False]]])

In [42]:
for idx_s, mtr_id in enumerate(mtr_id_list):
    
    print(mtr_id)
    print(metrics_square[idx_s])
    print(metrics_rect[idx_s])

005
[[ 0.89118162  0.9710165   0.86752419 33.19305799]
 [ 0.57763716  0.91301724  0.70780414 30.83077188]]
[[ 0.8927328   0.97250949  0.86802412 33.30277828]
 [ 0.60679205  0.9195634   0.70835717 31.20488442]]
006
[[ 0.89598112  0.97396702  0.87371292 33.43668824]
 [ 0.77559991  0.96024308  0.84264108 34.80783533]]
[[ 0.90811118  0.97720172  0.88886506 33.79967315]
 [ 0.81692077  0.9654837   0.85961073 35.45608601]]
030
[[ 0.8796219   0.97784509  0.88509669 34.85960084]
 [ 0.73243082  0.96131248  0.84339262 35.40080961]]
[[ 0.88413535  0.98106572  0.90560929 35.40313779]
 [ 0.7353292   0.96471274  0.85475883 35.67144364]]
034
[[ 0.91628186  0.97354171  0.87463565 33.65817557]
 [ 0.80586251  0.95205645  0.82532896 34.43914388]]
[[ 0.90331316  0.97843054  0.88977237 34.13679593]
 [ 0.80383745  0.96183994  0.84734332 35.1720652 ]]
048
[[ 0.91875133  0.9712345   0.8585076  33.23861965]
 [ 0.68384978  0.91087542  0.70782937 30.20192101]]
[[ 0.91104326  0.97326936  0.85935268 33.25782159]
 [

# OLD

### remove `_dc` from filenames

In [ ]:
path_data = '/bmrNAS/people/dvv/out_qdess/'

dir_list = [x[0] for x in os.walk(path_data)]

for d in dir_list:
    
    d = d + '/'
    file_list = get_file_list(d)
    
    if len(file_list) > 0:
        
        for f in file_list:
            
            if '_dc' not in f:
                continue
                            
            f_new = f.replace('_dc', '')
            
            arr = np.load(d + f)
            np.save(d + f_new, arr)
            
            os.remove(d + f)   